In [1]:
import os
"""
sets overrides the parent directory as current directory
"""
# Get the current directory path
current_directory = os.getcwd()

# Get the parent directory path
parent_directory = os.path.dirname(current_directory)

# Change the current directory to the parent directory
os.chdir(parent_directory)

# Verify the change
new_directory = os.getcwd()
print("New directory path:", new_directory)


# Now you can import modules from the parent directory
# from parent_module import my_function
from src.copperhead_processor import EventProcessor
from src.lib.get_parameters import getParametersForYr

import awkward as ak
import numpy as np
from omegaconf import OmegaConf
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

New directory path: /work/users/yun79/valerie/fork/copperheadV2


/work/users/yun79/valerie/fork/copperheadV2/src/copperhead_processor.py:20: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [2]:
param_path = "./configs/parameters/"
config = getParametersForYr(param_path, "2018")
coffea_processor = EventProcessor(config)

In [ ]:
load_path = "./test/stage1_inputs/dy_M-100To200"

dataset_dict = OmegaConf.load(load_path + "/dataset_dict.yaml")
dataset_dict = OmegaConf.to_object(dataset_dict) # convert to normal dictionary, which may not be necessary, but oh well
events = NanoEventsFactory.from_root(
    dataset_dict["files"],
    schemaclass=NanoAODSchema,
    metadata= dataset_dict["metadata"],
    uproot_options={
        "timeout":2400,
        # "allow_read_errors_with_report": True, # this makes process skip over OSErrors
    },
).events()
out_collections = coffea_processor.process(events)
out_collections = ak.zip(out_collections, depth_limit=1).compute()

load_path = "./test/stage1_outputs/dy_M-100To200"
target_out = ak.from_parquet(load_path+"/part0.parquet")
for field in target_out.fields:
    target_arr = ak.fill_none(target_out[field], -99.0)
    out_arr = ak.fill_none(out_collections[field], -99.0)
    is_same_arr = ak.all(ak.isclose(target_arr,out_arr))
    if not is_same_arr:
        print(f"ERROR: not same array for field {field}!")
        raise ValueError
print("Success!")

dataset: dy_M-100To200
events.metadata: {'sumGenWgts': 7720081.838819998, 'nGenEvts': 2000000, 'data_entries': None, 'fraction': 1.0, 'original_fraction': 1.0, 'is_mc': True, 'dataset': 'dy_M-100To200', 'NanoAODv': 9, 'version': 'latest'}
NanoAODv: 9
doing rochester!
doing fsr!
doing geofit!
electron_id: mvaFall17V2Iso_WP90
small file test sumWeights: 6798350.261352539
muons mass_resolution dpt1: dask.awkward<multiply, npartitions=1>
prepare jets NanoAODv: 9


awkward.highlevel.Array